# 01 - LeeQ Basics: Introduction to Concepts and Simulation

This notebook provides an introduction to LeeQ concepts and demonstrates basic simulation capabilities.

## Learning Objectives
- Understand core LeeQ concepts
- Learn about quantum element abstractions
- Practice with simulation backends
- Explore Chronicle logging integration

## Prerequisites
- Basic understanding of quantum computing
- Python programming knowledge

## Setup and Imports

First, let's import the necessary LeeQ modules and set up our simulation environment.

LeeQ is organized into several key modules:
- `leeq.core`: Core abstractions for quantum elements and operations
- `leeq.setups`: Hardware and simulation backend configurations
- `leeq.experiments`: Pre-built and custom experiment implementations
- `leeq.chronicle`: Experiment logging and data persistence
- `leeq.theory`: Quantum simulation and theoretical tools

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# Core LeeQ imports
from leeq.chronicle import Chronicle, log_and_record
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.experiments.experiments import ExperimentManager
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon

print("LeeQ modules imported successfully!")
print(f"NumPy version: {np.__version__}")
print("\nModules imported:")
print("  - Chronicle: Experiment logging and data persistence")
print("  - TransmonElement: Superconducting qubit representation")
print("  - HighLevelSimulationSetup: Fast phenomenological simulation")
print("  - ExperimentManager: Coordinates experiment execution")
print("  - VirtualTransmon: Simulated transmon for testing")

## Chronicle Integration

LeeQ uses Chronicle for experiment logging and data persistence. Chronicle automatically tracks:
- Experiment parameters and configurations
- Measurement data and results
- Analysis outputs and plots
- Calibration history and evolution

This creates a complete audit trail for reproducible quantum experiments.

In [ ]:
# Start Chronicle logging - this creates a timestamped log directory
import os
from datetime import datetime

# Clear any existing Chronicle instance and start fresh
try:
    Chronicle().stop_log()
except:
    pass

Chronicle().start_log()

# Chronicle creates structured logs that can be searched and analyzed
log_dir = Chronicle().get_log_dir()
print("Chronicle logging started!")
print(f"Log directory: {log_dir}")
print(f"Log timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Example of manual logging with Chronicle
log_and_record("tutorial_start", {
    "notebook": "01_basics",
    "user": "tutorial",
    "timestamp": datetime.now().isoformat(),
    "purpose": "Learning LeeQ basics and simulation"
})
print("\nTutorial start event logged to Chronicle")
print("You can find logs at: " + os.path.abspath(log_dir))

## Core LeeQ Concepts

### 1. Quantum Elements

In LeeQ, quantum systems are represented as **Quantum Elements**. The most common type is the `TransmonElement`, which represents a superconducting transmon qubit.

Key properties of quantum elements:
- **Frequency**: The qubit transition frequency
- **Anharmonicity**: Energy level spacing differences
- **Coherence times**: T1 (relaxation) and T2 (dephasing)
- **Control parameters**: Drive amplitudes, phases, and timings

In [ ]:
# Let's examine the VirtualTransmon class used in simulations
virtual_qubit = VirtualTransmon(
    name="DemoQubit",
    qubit_frequency=5.0,  # GHz
    anharmonicity=-0.2,   # GHz (negative for transmons)
    t1=50,                # microseconds
    t2=25,                # microseconds
    readout_frequency=9.0 # GHz
)

print(f"Virtual qubit created: {virtual_qubit.name}")
print(f"Qubit frequency: {virtual_qubit.qubit_frequency} GHz")
print(f"Anharmonicity: {virtual_qubit.anharmonicity} GHz")
print(f"T1: {virtual_qubit.t1} μs")
print(f"T2: {virtual_qubit.t2} μs")

# Log the qubit creation
log_and_record("qubit_created", {
    "name": virtual_qubit.name,
    "frequency": virtual_qubit.qubit_frequency,
    "t1": virtual_qubit.t1,
    "t2": virtual_qubit.t2
})

### 2. Simulation Backends

LeeQ provides several simulation backends:

- **High-Level Simulation**: Fast, phenomenological modeling for algorithm development
- **NumPy Backend**: Matrix-based quantum simulation for moderate system sizes
- **QuTiP Backend**: Advanced quantum dynamics and master equation solving

For learning and development, we'll use the high-level simulation which provides realistic noise models without the computational overhead.

In [ ]:
# Create an experiment manager to coordinate our experiments
manager = ExperimentManager()
manager.clear_setups()  # Clear any existing setups

# Create a high-level simulation setup with our virtual qubit
simulation_setup = HighLevelSimulationSetup(
    name='BasicTutorialSetup',
    virtual_qubits={1: virtual_qubit}  # Assign qubit to logical channel 1
)

# Register the setup with the experiment manager
manager.register_setup(simulation_setup)

print(f"Simulation setup created: {simulation_setup.name}")
print(f"Virtual qubits: {list(simulation_setup.virtual_qubits.keys())}")

# Log the setup creation
log_and_record("simulation_setup", {
    "setup_name": simulation_setup.name,
    "qubit_channels": list(simulation_setup.virtual_qubits.keys())
})

### 3. Experiment Framework

LeeQ experiments follow a structured pattern:

1. **Definition**: Specify what measurements to perform
2. **Compilation**: Convert high-level operations to hardware pulses
3. **Execution**: Run the experiment on hardware or simulation
4. **Analysis**: Process and visualize results

Let's demonstrate this with a simple state preparation and measurement.

In [ ]:
# Import basic experiments
from leeq.experiments.builtin.basic.calibrations import MeasurementStatistics
from leeq.experiments.builtin.basic.calibrations import RabiAmplitudeCalibration

# Create a measurement statistics experiment
# This measures the qubit in its ground state multiple times
measurement_exp = MeasurementStatistics(
    name="BasicMeasurement",
    qubit=1,  # Use logical qubit channel 1
    repeated_measurement_count=1000  # Take 1000 measurements
)

print(f"Experiment created: {measurement_exp.name}")
print(f"Target qubit: {measurement_exp.qubit}")
print(f"Measurement count: {measurement_exp.repeated_measurement_count}")

# Log the experiment setup
log_and_record("experiment_setup", {
    "experiment_type": "MeasurementStatistics",
    "qubit": measurement_exp.qubit,
    "measurement_count": measurement_exp.repeated_measurement_count
})

### 4. Running Your First Experiment

Now let's run our first experiment! This will measure the qubit in its ground state and show us the measurement statistics. Even in the ground state, we might see some excited state population due to:

- **Thermal population**: Finite temperature effects
- **Measurement errors**: Imperfect state discrimination
- **Relaxation during measurement**: T1 effects during readout

In [ ]:
# Run the experiment
print("Running basic measurement experiment...")
results = measurement_exp.run()

# Extract and display results
ground_state_prob = results['statistics']['ground_state_probability']
excited_state_prob = results['statistics']['excited_state_probability']
measurement_fidelity = results['statistics']['measurement_fidelity']

print(f"\nMeasurement Results:")
print(f"Ground state probability: {ground_state_prob:.3f}")
print(f"Excited state probability: {excited_state_prob:.3f}")
print(f"Measurement fidelity: {measurement_fidelity:.3f}")

# Create a simple visualization
states = ['Ground |0⟩', 'Excited |1⟩']
probabilities = [ground_state_prob, excited_state_prob]
colors = ['blue', 'red']

plt.figure(figsize=(8, 6))
bars = plt.bar(states, probabilities, color=colors, alpha=0.7)
plt.ylabel('Probability')
plt.title('Qubit Ground State Measurement Statistics')
plt.ylim(0, 1)

# Add probability labels on bars
for bar, prob in zip(bars, probabilities):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{prob:.3f}', ha='center', va='bottom')

plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Log the results
log_and_record("experiment_results", {
    "ground_state_prob": ground_state_prob,
    "excited_state_prob": excited_state_prob,
    "measurement_fidelity": measurement_fidelity
})

### 5. Understanding Quantum Noise

The simulation includes realistic noise models. Let's explore how different noise parameters affect our measurements by creating qubits with different coherence times.

In [ ]:
# Create qubits with different T1 times to see the effect of relaxation
qubit_configs = [
    {"name": "HighQuality", "t1": 100, "t2": 50, "color": "green"},
    {"name": "MediumQuality", "t1": 50, "t2": 25, "color": "orange"},
    {"name": "LowQuality", "t1": 10, "t2": 5, "color": "red"}
]

results_comparison = []

for config in qubit_configs:
    # Create virtual qubit with specific coherence times
    test_qubit = VirtualTransmon(
        name=config["name"],
        qubit_frequency=5.0,
        anharmonicity=-0.2,
        t1=config["t1"],
        t2=config["t2"],
        readout_frequency=9.0
    )
    
    # Create new setup for this qubit
    test_setup = HighLevelSimulationSetup(
        name=f'Setup_{config["name"]}',
        virtual_qubits={1: test_qubit}
    )
    
    # Create and run measurement experiment
    manager_test = ExperimentManager()
    manager_test.register_setup(test_setup)
    
    measurement_exp = MeasurementStatistics(
        name=f"Measurement_{config['name']}",
        qubit=1,
        repeated_measurement_count=1000
    )
    
    results = measurement_exp.run()
    
    results_comparison.append({
        "name": config["name"],
        "t1": config["t1"],
        "t2": config["t2"],
        "ground_prob": results['statistics']['ground_state_probability'],
        "fidelity": results['statistics']['measurement_fidelity'],
        "color": config["color"]
    })
    
    print(f"{config['name']} (T1={config['t1']}μs): Ground state prob = {results['statistics']['ground_state_probability']:.3f}")

# Visualize the comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot ground state probabilities
names = [r['name'] for r in results_comparison]
ground_probs = [r['ground_prob'] for r in results_comparison]
colors = [r['color'] for r in results_comparison]
t1_times = [r['t1'] for r in results_comparison]

ax1.bar(names, ground_probs, color=colors, alpha=0.7)
ax1.set_ylabel('Ground State Probability')
ax1.set_title('Effect of T1 on Ground State Measurement')
ax1.set_ylim(0.6, 1.0)

# Add T1 labels
for i, (name, prob, t1) in enumerate(zip(names, ground_probs, t1_times)):
    ax1.text(i, prob + 0.01, f'T1={t1}μs', ha='center', va='bottom')

# Plot T1 vs ground state probability
ax2.scatter(t1_times, ground_probs, c=colors, s=100, alpha=0.7)
ax2.set_xlabel('T1 Relaxation Time (μs)')
ax2.set_ylabel('Ground State Probability')
ax2.set_title('T1 vs Measurement Quality')
ax2.grid(alpha=0.3)

# Add trend line
z = np.polyfit(t1_times, ground_probs, 1)
p = np.poly1d(z)
ax2.plot(t1_times, p(t1_times), "--", alpha=0.5, color='gray')

plt.tight_layout()
plt.show()

# Log the comparison results
log_and_record("noise_comparison", {
    "qubit_configs": qubit_configs,
    "results": results_comparison
})

## Key Takeaways

From this basic introduction, you've learned:

1. **LeeQ Architecture**: Core modules and their responsibilities
2. **Chronicle Logging**: Automatic experiment tracking and data persistence
3. **Quantum Elements**: How qubits are represented and configured
4. **Simulation Backends**: High-level simulation for rapid development
5. **Experiment Framework**: Structure for defining and running experiments
6. **Noise Effects**: How coherence times affect measurement quality

### Next Steps

- **Practice**: Try modifying the qubit parameters and observe the effects
- **Explore**: Look at the Chronicle logs in the generated log directory
- **Learn More**: Continue to [02_single_qubit.ipynb](02_single_qubit.ipynb) for single qubit experiments

### Additional Resources

- LeeQ Documentation: Comprehensive API reference
- Example Notebooks: More specialized demonstrations
- Workflow Notebooks: Complete experimental procedures

In [ ]:
# Clean up and finalize logging
log_and_record("tutorial_complete", {
    "notebook": "01_basics",
    "experiments_run": ["MeasurementStatistics", "NoiseComparison"],
    "key_concepts": ["quantum_elements", "simulation", "chronicle", "noise_models"]
})

print("\n" + "="*50)
print("Tutorial 01 - LeeQ Basics completed successfully!")
print("Next: 02_single_qubit.ipynb")
print("="*50)